Progetto: **Classificazione di testi descrittivi per destinazione d'uso, inerenti ai beni sottoposti ad aste giudiziarie italiane.**

Studente: **Alessandro Monolo** | *10439147*

Relatore: Marco Brambilla

Referente aziendale: Simone Redaelli

Master: Data Science & Artificial Intelligence

Università: Politecnico di Milano

<hr style="border:1px solid black">

## Pre-Processing sulla variabile testuale per il modello XLM RoBERTa

- **1.** **<u>Pre-Processing sulla variabile testuale:</u>**
    - Per la fase di pre-processing sulla variabile testuale invece, partidirettamente da una base testuale pulita dai caratteri strani ed poi ho eseguito i seguenti steps per la pulizia del testo, togliendo:
        - **1.1** **e-mails**;
        - **1.2** **URLs**;
        - **1.3** **Doppi spazi o spazi multipli**;
        - **1.4** **Lower casa only**;
        - **1.5** **Caratteri accentati**;
        - **1.6** **Di nuovo - Doppi spazi o spazi multipli**;
        - **1.7** **Rimozione delle parole troppo corte, sotto i 2 caratteri**;
        - **1.8** **Elimino tutte le parole che presentano numeri in esse**, **ERRATO**
        - **1.9** **Rimozione parole non comuni**;
        

- **2.** **<u>Fase di Pre-Processign per la Target Variable "Destinazione d'uso", applicando LabelEncoder</u>**;


- **3.** **<u>Filtro il data frame</u>**;


- **5.** **<u>Export final CSV</u>**

<hr style="border:1px solid black">

**Importo le librerie che mi servono:**

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
import math
import plotly.express as px
import warnings
from matplotlib import cm
from PIL import Image
import string
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
import spacy
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

#### Set pandas options:

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 1000
pd.options.display.float_format = '{:.4f}'.format
pd.options.mode.chained_assignment = None

**Importo file CSV**

In [3]:
df = pd.read_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\10_Dataset_Pre_Processing_Var_Text\\Dataset_Pre_Processing_Var_Text.csv")

### 1 - Text Pre-Processing

- **1.1 Trovo tutte le emails nel campo descrizione e le elimino dal testo:**

In [4]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Clean"].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+._-]+)', '', x))

- **1.2 Trovo tutti i links/URL/Siti Web nel campo descrizione e li elimino dal testo:**

In [5]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w•,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))

- **1.3 Elimino tutti gli spazi doppi o spazi multipli nel campo Descrizione del bene:**

In [6]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: ' '.join(x.split()))

- **1.4 Converto tutte le parole presenti nel campo descrizione del bene in lower case only:**

In [7]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].str.lower()

- **1.5 Elimino tutti i caratteri che presentano accento:**


- Importante da considerare come questa fase di pulizia, sia necessaria per eliminare tutte quelle lettere che potrebbero deformare una parola o renderla malinterpretabile durante la fase di predizione.

- **1.8 Elimino tutte le parole che presentano numeri in esse:**

In [8]:
def remove_words_with_cypher(text):
    return ' '.join(word for word in text.split() if not re.search(r'\w*\d\w*', word))

# Apply the function to the column
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(remove_words_with_cypher)

- **1.6 Elimino di nuovo tutti gli spazi doppi o spazi multipli nel campo Descrizione del bene:**

In [9]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: ' '.join(x.split()))

- **1.7** **Elimino le parole troppo corte, sotto i due caratteri:**

- **1.9** **Elimino le parole meno frequenti che rappresentano errori di battitura o errori di merge tra parole:**

In [10]:
# Metto in una variabile tutte le stringhe presenti nella colonna:
text = " ".join(df["Descrizione_Bene_Pre_Process"])
text = text.split()
# Conto le stringhe contenute e le metto in una Pandas Series:
freq_common = pd.Series(text).value_counts()
# Prendo le 200 parole meno frequenti cge rappresentano tutte parole o numeri di nessun significato:
rare_words = freq_common.tail(500)

# Applico il drop alla colonna testuale pre-processata:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: " ".join([t for t in x.split() if t not in rare_words]))

### 2 - Fase di Pre-Processign per la Target Variable "Destinazione d'uso", applicando LabelEncoder

In [11]:
# Instanzio in una nuova variabile il modello per l'encoding della target variable:
label_encoder = LabelEncoder()

# Applico il modello di encoding alla mia target variable e salvo il risultato in una nuova colonna:
df['Destinazione_Uso_Encoded'] = label_encoder.fit_transform(df['Destinazione_Uso'])

### 3 - Filtro il data frame con solo le colonne che mi servono per la text classification

In [12]:
# Seleziono le colonne da eliminare
columns_to_drop = ['Destinazione_Uso', 
                   'Categoria_Catastale', 
                   'Tribunale', 
                   'Provincia', 
                   'Comune', 
                   'Numero_Lotto', 
                   'Numero_Vani', 
                   'Superficie', 
                   'Rapporto_Vani_Superficie', 
                   'N_Caratteri',
                   'N_Parole', 
                   'avg_word_len',
                   'Catasto_Fabbricati', 
                   'Descrizione_Bene',
                   'Descrizione_Bene_Clean', 
                   'Descrizione_Bene_EDA']
# Mantengo solo le colonne che mi servono per la text classification:
df = df.drop(columns=columns_to_drop)

### 5 - Export final CSV

In [13]:
df_csv = df.to_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\12_Dataset_XLM_RoBERTa_Model\\Dataset_XLM_RoBERTa_Model.csv", 
                   index=False)